# Time evolution

In [1]:
using ITensors
using ITensorNetworks
using ITensorNetworks: applyexp
using Observers
using Random

In this notebook we will go through some examples of simulating time evolution for a spin model on a lattice with a tree-like geometry using the [TDVP](https://arxiv.org/abs/1408.5056) algorithm.

As an example, we will consider a next-to-nearest neighbor spin-1/2 Heisenberg model with Hamiltonian

$$
H = J_1 \sum_{\langle i, j \rangle} \vec{S}_i \cdot \vec{S}_j + J_2 \sum_{\langle\langle i, j \rangle\rangle} \vec{S}_i \cdot \vec{S}_j + h \sum_i S^z_i,
$$

on a spin system defined on a small comb tree with 6 sites
<center><img align="center" width="200" src="./fig/small_comb_tree.svg"></center>

### Basic time evolution

We'll start by running a two-site TDVP scheme for a total time $t = 1$. This will involve $N = t / \tau$ sweeps, each of which evolves the given state by a time step $\tau = 0.1$.

In [2]:
# set TDVP parameters
cutoff = 1e-12
tau = 0.1
ttotal = 1.0

# define geometry and Hamiltonian
root_vertex = (3, 2)
c = named_comb_tree((3, 2))
s = siteinds("S=1/2", c)
Hos = ITensorNetworks.heisenberg(c)
H = TTN(Hos, s);

We can also keep track of some relevant quantities during each time evolution step using the ITensors.jl [observer system](https://itensor.github.io/ITensors.jl/dev/Observer.html#observer). Here we'll keep track of the local magnetization on the central site of the comb, as well as the total energy of the state.

In [3]:
using Observers

c = (2, 1)
obs = Observer(
    "Sz" => (; psi) -> expect("Sz", psi; vertices=[c])[c],
    "En" => (; psi) -> real(inner(psi', H, psi)),
);

We can now time evolve a given initial state state $|\phi\rangle$ over a time $t$ by calling `ψ = tdvp(H, -im * t, ϕ)`, which approximates $|\psi\rangle = \exp(-iHt)|\phi\rangle$.

In [4]:
ϕ = TTN(s, v -> iseven(sum(isodd.(v))) ? "Up" : "Dn") # start from product state
ψ1 = tdvp(
    H,
    -im * ttotal,
    ϕ;
    time_step=-im * tau,
    cutoff,
    normalize=false,
    (sweep_observer!)=obs
);

The magnetization and energy during at each time step can be read off from the observer:

In [5]:
obs

Row,Sz,En
,Complex…,Float64
1,-0.492534-1.70024e-20im,-1.25
2,-0.470545+3.44243e-20im,-1.25
3,-0.435229+6.50984e-19im,-1.25
4,-0.388492-1.29753e-18im,-1.25
5,-0.332825+2.07511e-18im,-1.25
6,-0.271141+4.63178e-18im,-1.25
7,-0.206587+5.17449e-18im,-1.25
8,-0.142354+3.51877e-18im,-1.25
9,-0.0814742+7.71132e-18im,-1.25


We can check some basic properties of this unitary time evolution, such as conservation of probability and conservation of energy.

In [6]:
@show norm(ψ1) ≈ 1.0
@show real(inner(ψ1', H, ψ1)) ≈ inner(ϕ', H, ϕ);

norm(ψ1) ≈ 1.0 = true
real(inner(ψ1', H, ψ1)) ≈ inner(ϕ', H, ϕ) = true


We can also perform the same evolution backwards and check that we end up close to the original state.

In [7]:
ψ2 = tdvp(
    H,
    +im * ttotal,
    ψ1;
    time_step=+im * tau,
    cutoff,
    normalize=false,
)
@show norm(ψ2) ≈ 1.0
@show abs(inner(ϕ, ψ2)) > 0.99;

norm(ψ2) ≈ 1.0 = true
abs(inner(ϕ, ψ2)) > 0.99 = true


### Sum of Hamiltonians

Time evolution also supports the use of Hamiltonians which are defined as a lazy sum of component Hamiltonians (as do all other sweeping routines in ITensorNetworks.jl for that matter). To evolve using a sum of Hamiltonians, we can just replace the `TTN` Hamiltonian operator by a `Vector{TTN}` which contains all Hamiltonian terms:

```julia
ψ = tdvp([H1, H2, ...], t, ϕ; kwargs...)
```

### Custom solvers

Instead of just supplying a Hamiltonian operator in `TTN` format to the TDVP routine, we can instead supply a custom local solver which encodes how a local region should be updated during the sweeping routine. As an example, we can run TDVP with a local solver that uses a custom exponentiation algorithm found in `ITensorNetworks.applyexp` instead of the default `Krylovkit.exponentiate` solver.

This generally requires some knowledge about the internals of the sweeping routines, which are not well documented at this time. Detailed instructions for defining custom solvers will be added to the ITensorNetworks.jl documentation soon. For now, some more details are given in the [example on time evolution for time-dependent problems](./tdvp_dynamic_demo.ipynb).

In [8]:
function solver(PH, ψ0; time_step, kwargs...)
    solver_kwargs = (;
        ishermitian=true, tol=1e-12, krylovdim=30, maxiter=100, verbosity=0, eager=true
    )
    psi, exp_info = applyexp(PH, time_step, ψ0; solver_kwargs...)
    return psi, (; info=exp_info)
end

ψ3 = tdvp(
    solver,
    H,
    -im * ttotal,
    ϕ;
    time_step=-im * tau,
    cutoff,
    normalize=false,
)

@show norm(ψ3) ≈ 1.0
@show real(inner(ψ3', H, ψ3)) ≈ inner(ϕ', H, ϕ)
@show inner(ψ1, ψ3) ≈ 1.0; # check if custom solver gives the same result

norm(ψ3) ≈ 1.0 = true
real(inner(ψ3', H, ψ3)) ≈ inner(ϕ', H, ϕ) = true
inner(ψ1, ψ3) ≈ 1.0 = true


true

### Imaginary time evolution

We can also use the TDVP routine for ground state searches by performing several sweeps of imaginary time evolution:

In [9]:
reverse_step = true
cutoff = 1e-12
tau = 1.0
ttotal = 50.0

ϕ = normalize!(random_ttn(s; link_space=2))
ψ = copy(ϕ)

trange = 0.0:tau:ttotal
for (step, t) in enumerate(trange)
    nsite = (step <= 10 ? 2 : 1)
    ψ = tdvp(
        H, -tau, ψ; cutoff, nsite, reverse_step=false, normalize=true, exponentiate_krylovdim=15
    )
end
e = real(inner(ψ', H, ψ))

-2.4747448713915907

We can compare the resulting ground state energy to the one we would obtain using DMRG:

In [10]:
ψ_dmrg = dmrg(H, ϕ; nsweeps=10, cutoff, nsite=2)
e_dmrg = real(inner(ψ_dmrg', H, ψ_dmrg))

-2.4747448713915907